In [1]:
from pathlib import Path
import sqlite3
import pandas as pd
import numpy as np
import os

# Get project root (go up one level from models/ folder)
BASE_DIR = Path(os.getcwd()).parent

# Load config dynamically
import importlib.util
spec = importlib.util.spec_from_file_location("config", BASE_DIR / "config.py")
config = importlib.util.module_from_spec(spec)
spec.loader.exec_module(config)

EMA_FAST = config.EMA_FAST
EMA_SLOW = config.EMA_SLOW

# Path to SQLite with EMA already computed
db_path = BASE_DIR / "data" / "processed" / "data_processed.sqlite"

# Load table
with sqlite3.connect(db_path) as conn:
    tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
    if tables.empty:
        raise RuntimeError(f"No tables found in {db_path}")
    table_name = tables["name"].iloc[0]
    df = pd.read_sql(f'SELECT * FROM "{table_name}"', conn, parse_dates=["Date"])

df = df.sort_values("Date").set_index("Date")

# Ensure required columns exist
for col in ["Close", f"EMA{EMA_FAST}", f"EMA{EMA_SLOW}"]:
    if col not in df.columns:
        raise KeyError(f"Required column '{col}' not found; columns = {df.columns.tolist()}")

# EMA strategy: signal = 1 if EMA_FAST > EMA_SLOW, else -1
df["signalEMA"] = np.where(df[f"EMA{EMA_FAST}"] > df[f"EMA{EMA_SLOW}"], 1, -1)

# Returns based on signal
df["market_ret"] = df["Close"].pct_change().fillna(0)
df["strategy_ret"] = df["signalEMA"].shift(1).fillna(0) * df["market_ret"]
df["strategy_eq"] = (1 + df["strategy_ret"]).cumprod()

print(df[["Close", f"EMA{EMA_FAST}", f"EMA{EMA_SLOW}", "signalEMA"]].tail())

# Save EMA signals to SQLite
out_db_path = BASE_DIR / "data" / "signals" / "EMAsignal.sqlite"
out_db_path.parent.mkdir(parents=True, exist_ok=True)

with sqlite3.connect(out_db_path) as conn:
    df.reset_index().to_sql("signals_ema", conn, if_exists="replace", index=False)

print("Saved EMA signals to:", out_db_path.resolve())

                 Close       EMA50      EMA200  signalEMA
Date                                                     
2025-12-15  274.109985  267.697340  240.202232          1
2025-12-16  274.609985  267.968424  240.544597          1
2025-12-17  271.839996  268.120250  240.855994          1
2025-12-18  272.190002  268.279848  241.167775          1
2025-12-19  273.670013  268.491227  241.491181          1
Saved EMA signals to: C:\Users\soltv\Documents\GitHub\newalgotrade\data\signals\EMAsignal.sqlite
